# Sprint 4 — Webinar 14 (Práctica)

**Modalidad:** Coding Together + Breakrooms  
**Tema:** Funnels, CTEs y escenarios de impacto (SQLite)  
**Versión:** 2025-11-14T10:00:00

> Esta sesión práctica parte de lo aprendido en **S4W13 (Teórico)**. Vamos a:
> - Construir un funnel de conversión con CTEs.
> - Simular mejoras en etapas específicas del funnel.
> - Traducir cambios de conversión en impacto de negocio.
> - Preparar recomendaciones accionables para stakeholders.


### Agenda – Webinar 14 (Práctico)

| Minuto | Actividad | Modalidad |
|--------|-----------|-----------|
| 0–10   | Contexto y repaso teórico breve | Plenaria |
| 10–30  | **Ejercicio 1:** Funnel base con CTEs | Coding Together |
| 30–35  | Puesta en común rápida | Plenaria |
| 35–55  | **Ejercicio 2:** Simular mejoras y medir uplift | Coding Together + Breakrooms |
| 55–75  | **Ejercicio 3:** Segmentos, tablero ejecutivo y recomendaciones | Breakrooms + Plenaria |
| 75–80  | Cierre + recursos para seguir practicando SQL | Plenaria |


## 0. Objetivos de aprendizaje

Al final de este webinar deberías ser capaz de:

1. **Construir** un funnel de conversión usando CTEs encadenadas en SQL.
2. **Calcular** tasas de conversión por etapa y por variante A/B.
3. **Simular** mejoras en etapas específicas y estimar el impacto en revenue.
4. **Comunicar** hallazgos de manera clara a stakeholders de negocio usando tablas/resúmenes.


## 0.1. Cómo usar este notebook con SQLite Online

En este webinar **no conectaremos Python a una base de datos**: usaremos la herramienta web `SQLiteOnline` (u otra similar).

Pasos recomendados:

1. Abre la página de tu preferencia para trabajar con **SQLite online**.  
2. Elige el motor **SQLite**.  
3. Copia y pega los bloques de código SQL marcados como:

   ```sql
   -- ejemplo
   SELECT 1;
   ```

4. Ejecuta cada bloque por separado.  
5. Si algo falla, revisa que:
   - Hayas corrido **antes** el bloque de creación de tablas (DDL).  
   - No hayas olvidado el `;` final.  
   - Estés en el motor correcto (SQLite).  

> **Tip docente:** puedes compartir el script DDL por chat o en la plataforma para que los estudiantes solo se concentren en escribir consultas SQL.


## 1. Recordatorio teórico — Funnels, CTEs y escenarios

Antes de entrar a los ejercicios, sintetizamos tres ideas clave del webinar teórico:

### 1.1 Funnels de conversión

- Un **funnel** describe la secuencia de pasos que sigue un usuario hasta completar una acción de negocio.  
- Ejemplo típico de e-commerce:
  1. `viewed_product`
  2. `added_to_cart`
  3. `begin_checkout`
  4. `purchased`

- Las métricas básicas por etapa son:
  - **Tasa de avance**: usuarios que pasan de una etapa a la siguiente.
  - **Tasa de conversión final**: usuarios que llegan a `purchased` / usuarios que inician el funnel.

### 1.2 CTEs para modelar funnels

- Un **CTE** (`WITH ... AS (...)`) permite construir pasos intermedios legibles.  
- Para funnels, es habitual tener un CTE por etapa, por ejemplo:

```sql
WITH viewed AS (
  SELECT session_id, user_id, variant
  FROM funnel_sessions
  WHERE viewed_product = 1
),
added AS (
  SELECT *
  FROM viewed
  WHERE session_id IN (
    SELECT session_id
    FROM funnel_sessions
    WHERE added_to_cart = 1
  )
),
...
```

- Encadenar CTEs ayuda a:
  - Evitar subconsultas anidadas difíciles de leer.
  - Reutilizar la misma lógica de conteo en diferentes consultas.

### 1.3 Escenarios de negocio

- Un cambio de producto (ej. nueva UI, nuevo copy) suele:
  - **Modificar** una o varias tasas de conversión.
  - Tener impacto **diferente** según el segmento (web vs mobile, país, etc.).  

- El objetivo de hoy: traducir estos cambios a números concretos de negocio usando SQL.


## 1.4 Script DDL de la sesión (ejecutar primero)

Ejecuta el siguiente bloque en tu motor SQLite online **antes** de empezar los ejercicios.

> Crea las tablas `users` y `funnel_sessions` y carga un dataset pequeño de ejemplo con variantes A/B y segmentos.


In [ ]:
-- ==============================================
-- DDL S4W14 para SQLiteOnline (sin esquemas)
-- Pega y ejecuta este bloque primero.
-- ==============================================

PRAGMA foreign_keys = ON;

DROP TABLE IF EXISTS funnel_sessions;
DROP TABLE IF EXISTS users;

CREATE TABLE users (
  user_id INTEGER PRIMARY KEY,
  segment TEXT NOT NULL,
  country TEXT NOT NULL
);

CREATE TABLE funnel_sessions (
  session_id INTEGER PRIMARY KEY,
  user_id INTEGER NOT NULL,
  variant TEXT NOT NULL,
  viewed_product INTEGER NOT NULL,
  added_to_cart INTEGER NOT NULL,
  begin_checkout INTEGER NOT NULL,
  purchased INTEGER NOT NULL,
  revenue REAL NOT NULL,
  session_date DATE NOT NULL,
  FOREIGN KEY (user_id) REFERENCES users(user_id)
);

INSERT INTO users (user_id, segment, country) VALUES
  (1, 'web', 'CO'),
  (2, 'mobile', 'CO'),
  (3, 'web', 'CO'),
  (4, 'web', 'AR'),
  (5, 'web', 'AR'),
  (6, 'mobile', 'CO'),
  (7, 'web', 'CO'),
  (8, 'web', 'CO'),
  (9, 'web', 'AR'),
  (10, 'web', 'AR'),
  (11, 'mobile', 'CO'),
  (12, 'mobile', 'AR');

INSERT INTO funnel_sessions (
  session_id, user_id, variant,
  viewed_product, added_to_cart, begin_checkout, purchased,
  revenue, session_date
) VALUES
  (1, 1, 'A', 1,1,1,1, 75.0, '2025-10-05'),
  (2, 1, 'A', 1,0,0,0, 0.0, '2025-10-11'),
  (3, 2, 'A', 1,1,1,1, 75.0, '2025-10-02'),
  (4, 3, 'A', 1,0,0,0, 0.0, '2025-10-13'),
  (5, 3, 'A', 1,1,0,0, 0.0, '2025-10-20'),
  (6, 4, 'A', 1,1,1,1, 75.0, '2025-10-03'),
  (7, 4, 'A', 1,0,0,0, 0.0, '2025-10-13'),
  (8, 5, 'B', 1,1,1,1, 60.0, '2025-10-09'),
  (9, 5, 'A', 1,1,1,0, 0.0, '2025-10-06'),
  (10, 6, 'B', 1,1,0,0, 0.0, '2025-10-18'),
  (11, 6, 'A', 1,1,0,0, 0.0, '2025-10-02'),
  (12, 7, 'A', 1,0,0,0, 0.0, '2025-10-13'),
  (13, 8, 'A', 1,1,0,0, 0.0, '2025-10-11'),
  (14, 8, 'A', 1,1,1,0, 0.0, '2025-10-15'),
  (15, 9, 'B', 1,1,0,0, 0.0, '2025-10-18'),
  (16, 10, 'B', 1,0,0,0, 0.0, '2025-10-13'),
  (17, 10, 'B', 1,1,0,0, 0.0, '2025-10-17'),
  (18, 11, 'A', 1,1,0,0, 0.0, '2025-10-05'),
  (19, 11, 'A', 1,1,1,1, 90.0, '2025-10-20'),
  (20, 12, 'B', 1,0,0,0, 0.0, '2025-10-01'),
  (21, 12, 'A', 1,1,1,1, 75.0, '2025-10-04');

## 2. Ejercicio 1 — Funnel base y tasas de conversión por variante

**Fundamento:** antes de simular mejoras, necesitamos entender **cómo está hoy** el funnel.

### Enunciado

Usando la tabla `funnel_sessions`:

1. Calcula, para cada `variant` (`A` y `B`):
   - Número total de sesiones que **inician** el funnel (`viewed_product = 1`).
   - Sesiones que llegan a cada etapa: `added_to_cart`, `begin_checkout`, `purchased`.
2. A partir de esos conteos, calcula las **tasas de conversión**:
   - `view → cart`
   - `cart → checkout`
   - `checkout → purchase`
   - `view → purchase` (conversión final).

### Requisitos técnicos

- Usa **CTEs encadenadas** para que el funnel quede legible.
- Devuelve una tabla final con una fila por `variant` y columnas como:

| variant | n_view | n_cart | n_checkout | n_purchase | conv_view_cart | conv_cart_checkout | conv_checkout_purchase | conv_view_purchase |

### Pistas

- Pista 1: parte de un CTE `base` con todas las sesiones que tienen `viewed_product = 1`.
- Pista 2: en cada CTE posterior, puedes filtrar las sesiones que avanzan a la siguiente etapa.
- Pista 3: para las tasas, recuerda que puedes usar `1.0 * numerador / denominador` para evitar división entera.
- Pista 4: si quieres redondear, puedes usar `ROUND(campo, 3)` para dejar 3 decimales.

### Comentarios para la docencia

- Primeros 10–15 minutos en **Coding Together**, escribiendo la estructura general.  
- Luego, 5–10 minutos en **Breakrooms** para que los estudiantes completen la consulta y comparen resultados.


In [ ]:
-- Ejercicio 1: embudo base por variante
-- Intenta resolverlo primero por tu cuenta.
-- Sugerencia: completa los '...' con la lógica correspondiente.

WITH base AS (
  SELECT
    session_id,
    variant,
    viewed_product,
    added_to_cart,
    begin_checkout,
    purchased
  FROM funnel_sessions
  WHERE viewed_product = 1
),
agg AS (
  SELECT
    variant,
    COUNT(*)                                  AS n_view,
    SUM(CASE WHEN added_to_cart  = 1 THEN 1 ELSE 0 END) AS n_cart,
    SUM(CASE WHEN begin_checkout = 1 THEN 1 ELSE 0 END) AS n_checkout,
    SUM(CASE WHEN purchased      = 1 THEN 1 ELSE 0 END) AS n_purchase
  FROM base
  GROUP BY variant
)
SELECT
  variant,
  n_view,
  n_cart,
  n_checkout,
  n_purchase,
  ROUND(1.0 * n_cart      / n_view,      3) AS conv_view_cart,
  ROUND(1.0 * n_checkout  / n_cart,      3) AS conv_cart_checkout,
  ROUND(1.0 * n_purchase  / n_checkout,  3) AS conv_checkout_purchase,
  ROUND(1.0 * n_purchase  / n_view,      3) AS conv_view_purchase
FROM agg
ORDER BY variant;


## 3. Ejercicio 2 — Simular una mejora en una etapa del funnel

**Fundamento:** los equipos de producto suelen decir cosas como:  
> "Si mejoramos el paso de checkout, podríamos subir la conversión final en 5 puntos porcentuales".

Vamos a cuantificar ese escenario.

### Enunciado

1. Partiendo de los resultados del **Ejercicio 1**, considera la variante `B` como la **nueva experiencia**.
2. Supón que logras mejorar la tasa `checkout → purchase` de la variante `B` en **+5 puntos porcentuales** (por ejemplo, de 0.50 a 0.55).
3. Asume que:
   - El número de sesiones que llegan a `begin_checkout` se mantiene constante.
   - El **ticket promedio** (`avg_revenue_per_purchase`) se mantiene igual.

4. Calcula:

   - Número actual de compras en `B` (`n_purchase_actual`).
   - Número simulado de compras en `B` (`n_purchase_sim`).
   - Revenue actual (`revenue_actual_B`).
   - Revenue simulado (`revenue_sim_B`).

### Requisitos técnicos

- Puedes resolverlo en dos pasos:
  1. Una consulta que compute las métricas actuales por variante (reutilizando tu consulta del Ejercicio 1 y añadiendo `avg_revenue_per_purchase`).
  2. Un CTE adicional que aplique la mejora al paso `checkout → purchase` y derive el revenue simulado.

### Pistas

- Pista 1: calcula `avg_revenue_per_purchase` como `SUM(revenue) / NULLIF(n_purchase, 0)` solo sobre sesiones con `purchased = 1`.
- Pista 2: el número simulado de compras será:  
  `n_purchase_sim = n_checkout * conv_checkout_purchase_mejorada`  
  donde `conv_checkout_purchase_mejorada = conv_checkout_purchase + 0.05`.
- Pista 3: el revenue simulado será:  
  `revenue_sim_B = n_purchase_sim * avg_revenue_per_purchase`.
- Pista 4: trabaja solo con la variante `B` en el último SELECT para simplificar.

### Comentarios para la docencia

- 10–15 minutos en **Coding Together** para armar los CTE base.  
- 10 minutos en **Breakrooms** para que cada grupo pruebe distintos valores de mejora (+3 p.p., +10 p.p., etc.) y discuta el impacto.


In [ ]:
-- Ejercicio 2: simulación de mejora en checkout -> purchase (variante B)

WITH funnel AS (
  SELECT
    fs.variant,
    COUNT(*) AS n_view,
    SUM(CASE WHEN fs.begin_checkout = 1 THEN 1 ELSE 0 END) AS n_checkout,
    SUM(CASE WHEN fs.purchased      = 1 THEN 1 ELSE 0 END) AS n_purchase,
    SUM(fs.revenue) AS revenue_total
  FROM funnel_sessions fs
  WHERE fs.viewed_product = 1
  GROUP BY fs.variant
),
metrics AS (
  SELECT
    variant,
    n_view,
    n_checkout,
    n_purchase,
    revenue_total,
    CASE WHEN n_purchase > 0
         THEN 1.0 * revenue_total / n_purchase
         ELSE 0.0 END AS avg_revenue_per_purchase,
    CASE WHEN n_checkout > 0
         THEN 1.0 * n_purchase / n_checkout
         ELSE 0.0 END AS conv_checkout_purchase
  FROM funnel
),
sim AS (
  SELECT
    variant,
    n_view,
    n_checkout,
    n_purchase    AS n_purchase_actual,
    revenue_total AS revenue_actual,
    avg_revenue_per_purchase,
    conv_checkout_purchase,
    -- Mejora de +5 puntos porcentuales
    conv_checkout_purchase + 0.05 AS conv_checkout_purchase_mejorada,
    -- Compras simuladas
    ROUND(n_checkout * (conv_checkout_purchase + 0.05)) AS n_purchase_sim
  FROM metrics
)
SELECT
  variant,
  n_view,
  n_checkout,
  n_purchase_actual,
  revenue_actual,
  avg_revenue_per_purchase,
  conv_checkout_purchase,
  conv_checkout_purchase_mejorada,
  n_purchase_sim,
  ROUND(n_purchase_sim * avg_revenue_per_purchase, 2) AS revenue_sim
FROM sim
WHERE variant = 'B';


## 4. Ejercicio 3 — Segmentos, tablero ejecutivo y recomendaciones

**Fundamento:** el mismo cambio de producto puede tener impactos muy distintos por segmento (ej. `web` vs `mobile`, `CO` vs `MX`).

### Enunciado

Usando las tablas `users` y `funnel_sessions`:

1. Calcula, por combinación de `segment` (web/mobile) y `variant` (A/B):
   - Sesiones que inician el funnel.
   - Sesiones que llegan a `purchased`.
   - Tasa de conversión final `view → purchase`.
   - Revenue total.

2. Organiza los resultados en una tabla pensada como **tablero ejecutivo**, por ejemplo:

| segment | variant | n_view | n_purchase | conv_view_purchase | revenue_total |

3. A partir de estos resultados, responde:

   - ¿En qué segmento y variante se observa la **mejor conversión**?  
   - ¿Cuál es el segmento que más **revenue total** aporta hoy?  
   - Si tuvieras que priorizar un segmento para lanzar primero la mejora del Ejercicio 2, ¿cuál sería y por qué?

### Pistas

- Pista 1: necesitarás un `JOIN` entre `funnel_sessions` (`fs`) y `users` (`u`) por `user_id`.
- Pista 2: limita el análisis a sesiones donde `viewed_product = 1`.
- Pista 3: agrupa por `segment` y `variant`.
- Pista 4: calcula la tasa de conversión como `n_purchase * 1.0 / n_view`.

### Comentarios para la docencia

- 5 minutos para modelar juntos la estructura general de la consulta.  
- 10–15 minutos en **Breakrooms** para que los grupos completen la query, obtengan resultados y preparen 2–3 bullets de recomendación.  
- 5–10 minutos finales para que algunos grupos presenten su lectura de negocio.


In [ ]:
-- Ejercicio 3: tablero ejecutivo por segmento y variante

WITH joined AS (
  SELECT
    u.segment,
    fs.variant,
    fs.viewed_product,
    fs.purchased,
    fs.revenue
  FROM funnel_sessions fs
  JOIN users u
    ON fs.user_id = u.user_id
  WHERE fs.viewed_product = 1
),
agg AS (
  SELECT
    segment,
    variant,
    COUNT(*) AS n_view,
    SUM(CASE WHEN purchased = 1 THEN 1 ELSE 0 END) AS n_purchase,
    SUM(revenue) AS revenue_total
  FROM joined
  GROUP BY segment, variant
)
SELECT
  segment,
  variant,
  n_view,
  n_purchase,
  ROUND(CASE WHEN n_view > 0
             THEN 1.0 * n_purchase / n_view
             ELSE 0.0 END, 3) AS conv_view_purchase,
  ROUND(revenue_total, 2) AS revenue_total
FROM agg
ORDER BY segment, variant;


## 7. Cierre y siguientes pasos

### 7.1 Qué deberías llevarte

De este webinar práctico deberías salir con:

- Mayor soltura para usar **CTEs** en análisis de funnels.  
- Una estrategia concreta para **simular escenarios** de mejora y cuantificar su impacto.  
- Un ejemplo de cómo construir un **tablero ejecutivo** simple con SQL (listas agregadas) antes de ir a herramientas de BI.  

### 7.2 Siguientes pasos sugeridos

Para seguir profundizando, puedes:

- Repetir los ejercicios cambiando:
  - La etapa que se mejora (ej. `view → cart` en vez de `checkout → purchase`).  
  - El segmento priorizado (`mobile` primero, luego `web`).  
  - El tamaño del uplift (+3 p.p., +7 p.p., +10 p.p.).  
- Añadir más atributos a `users` (ej. tipo de plan, canal de adquisición) y extender el tablero ejecutivo.  
- Llevar los resultados a **Google Sheets** o a una herramienta de BI y armar gráficos que comparen escenarios.

### 7.3 Recursos recomendados para seguir estudiando SQL y funnels

**SQL (general):**
- "Practical SQL" – Anthony DeBarros.  
- Tutoriales interactivos tipo *SQL sandbox* (por ejemplo, plataformas que ofrecen ejercicios guiados con corrección automática).  
- Documentación oficial de SQLite (tipos de datos, funciones de agregación, cláusulas `WITH`).  

**Análisis de funnels y producto digital:**
- Blogs de analítica de producto (ej. artículos sobre funnels, AARRR, experimentación).  
- Material de A/B testing básico (p-values, intervalos de confianza, test de proporciones).  
- Casos de estudio de e-commerce donde se analicen mejoras en embudos de conversión.

> Puedes adaptar o ampliar esta sección con recursos propios de la escuela/plataforma (notas de clase, vídeos, repositorios, etc.).
